In [ ]:
!pip install mtcnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.5 MB/s eta 0:00:00


In [ ]:
import cv2
import dlib
import matplotlib.pyplot as plt
from mtcnn import MTCNN

In [ ]:
from google.colab import files

In [ ]:
# Load the MTCNN face detection model
mtcnn = MTCNN()

In [11]:
# Function to detect skin regions in an image
def skin_detection(image):
    # Convert the image to YCrCb color space
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # Define lower and upper thresholds for skin color in YCrCb space
    lower = np.array([0, 135, 85], dtype=np.uint8)
    upper = np.array([255, 180, 135], dtype=np.uint8)

    # Create a binary mask to detect skin regions
    mask = cv2.inRange(ycrcb, lower, upper)

    # Apply morphological operations to remove noise
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
    mask = cv2.dilate(mask, kernel, iterations=1)

    return mask

In [12]:
# Function to process skin regions in an image
def process_skin_regions(image):
    # Detect skin regions in the image
    skin_mask = skin_detection(image)

    # Extract the skin regions using the mask
    skin_regions = cv2.bitwise_and(image, image, mask=skin_mask)

    return skin_regions

In [ ]:
def detect_and_process_skin(image_path, output_folder):
    # Read the input image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Extract the class label from the image path
    class_label = os.path.basename(os.path.dirname(image_path))

    # Create the class output folder if it doesn't exist
    class_output_folder = os.path.join(output_folder, class_label)
    os.makedirs(class_output_folder, exist_ok=True)

    # Detect faces in the image
    detections = mtcnn.detect_faces(img_rgb)

    # Check if any faces were detected
    if detections:
        # Extract the first detected face
        x, y, width, height = detections[0]['box']

        # Crop the face region
        face_img = img[y:y+height, x:x+width]

        # Process skin regions within the cropped face image
        processed_skin_image = process_skin_regions(face_img)

        # Save the processed skin image in the class output folder
        output_skin_path = os.path.join(class_output_folder, f"processed_skin_{os.path.basename(image_path)}")
        cv2.imwrite(output_skin_path, processed_skin_image)
        print(f"Processed skin image saved successfully: {output_skin_path}")
        return True
    else:
        print(f"No faces were detected in the image: {image_path}")
        # If no faces are detected, extract skin regions or save the original image
        skin_mask = skin_detection(img)
        if np.any(skin_mask):
            # Extract skin regions using the mask
            skin_regions = cv2.bitwise_and(img, img, mask=skin_mask)
            output_skin_path = os.path.join(class_output_folder, f"processed_skin_{os.path.basename(image_path)}")
            cv2.imwrite(output_skin_path, skin_regions)
            print(f"Processed skin image saved successfully: {output_skin_path}")
        else:
            # Save the original image if no skin regions are detected
            output_original_path = os.path.join(class_output_folder, f"original_{os.path.basename(image_path)}")
            cv2.imwrite(output_original_path, img)
            print(f"No skin regions detected. Original image saved: {output_original_path}")
        return False


In [ ]:
# Function to process a set of images
def process_images(image_paths, output_folder):
    for i, image_path in enumerate(image_paths):
        detect_and_process_skin(image_path, output_folder)

# Path to the input folder in Google Drive
input_folder = '/content/drive/My Drive/Dataset'
# Path to the output folder in Google Drive
output_folder = '/content/drive/My Drive/Processed_Images'

# List files in the input folder
import os
image_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Process the images
process_images(image_files, output_folder)